In [ ]:
import requests
import numpy as np
!pip install "tensorflow>=1.14.0,<2.0"
!pip install "tensorflow-gpu>=1.14.0,<2.0"
!pip install  ampligraph

In [3]:
import ampligraph
import numpy as np
ampligraph.__version__
import pandas as pd
import csv
from ampligraph.datasets import load_from_ntriples
from ampligraph.datasets import load_from_csv

In [4]:
Y_train=load_from_csv('/content','1100train.csv',sep=',',add_reciprocal_rels=False)

In [5]:
Y_test=load_from_csv('/content','1000test.csv',sep=',',add_reciprocal_rels=False)

In [19]:
print(Y_train)

[['Common symptoms' 'include' 'fever']
 ['Management' 'involves' 'treatment']
 ['The World Health Organization' 'declared' 'outbreak']
 ...
 ['Recent studies' 'shown' 'Nocardia infections']
 ['individuals' 'increased' 'rates']
 ['autoimmune diseases' 'have' 'association']]


In [6]:
Y_train.shape

(1053, 3)

In [21]:
entities = np.unique(np.concatenate([Y_train[:, 0], Y_train[:, 2]]))
entities.size

1792

In [22]:
Y_test.shape

(25, 3)

In [9]:
from ampligraph.latent_features import DistMult

In [10]:
model = DistMult(batches_count=10, 
                seed=0, 
                epochs=20, 
                k=10, 
                eta=10,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [11]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(Y_train, early_stopping = False)

Average Loss:   2.368018: 100%|██████████| 20/20 [00:00<00:00, 27.00epoch/s]


In [26]:
relations = np.unique(Y_train[:, 1])
relations.size

489

In [30]:
H1=model.get_embeddings(['Common symptoms'],embedding_type='entity')
type(H1)
H1

array([[ 0.08750542,  0.12748615, -0.10542529, -0.11433046, -0.13527544,
         0.11033006,  0.00808538,  0.04920774, -0.06920921,  0.12361796]],
      dtype=float32)

In [35]:
H1

array([[ 0.07764134, -0.14123002,  0.10496614,  0.1007892 ,  0.09594934,
         0.08170252, -0.03605187, -0.02447914, -0.0216195 , -0.08576715]],
      dtype=float32)

In [77]:
import pandas as pd
from sklearn import preprocessing

from statistics import mean
data = pd.read_csv('1100train.csv',header=None)
data.head()
arr =[]

type(data)

for i,j,k in zip(data[0],data[1],data[2]):
  print(i,j,k)
  H1=model.get_embeddings([i],embedding_type='entity')
  T1=model.get_embeddings([k],embedding_type='entity')
  R1=model.get_embeddings([j], embedding_type='relation')
  df3=np.add(H1,R1)-T1
  normalized = preprocessing.normalize(df3)
  # print("Normalized Data = ", normalized)
  ar=abs(normalized)
  # print(ar)
  # break
  for i in ar:
    Score=mean(i)
    # print(Score)
    arr.append(Score)

print(arr)




Common symptoms include fever
Management involves treatment
The World Health Organization declared outbreak
common symptoms include cough
minority develop noticeable symptoms
Cardiovascular complications include heart failure
Sputum carry large amounts
indicative suggest underlying immunopathology
people have classical serum biomarkers
The US Food and Drug Administration approved test
guidelines recommend medication
Face coverings limit volume
the United States Environmental Protection Agency maintains list
Intensivists compiled treatment recommendations
availability affect mortality
study reported earliest date
Official publications reported earliest onset
Wuhan Central Hospital sent bronchoalveolar lavage fluid BAL sample
the National Health Commission of China issued notice
China reported nearly 140 new cases
Italy overtook China
the United States overtaken China
ratio reflects number
the World Health Organization reported estimates
random antibody testing suggested national IFR
Ear

In [76]:
import pandas
df = pandas.DataFrame(data={"col1": arr})
df.to_csv("./scores.csv", sep=',',index=False)

In [73]:
type(Score)

numpy.float32

0                          Management
1       The World Health Organization
2                     common symptoms
3                            minority
4        Cardiovascular complications
                    ...              
1047                     Lung disease
1048                           5 10 %
1049                   Recent studies
1050                      individuals
1051              autoimmune diseases
Name: Common symptoms, Length: 1052, dtype: object

In [16]:
H1

array([[ 0.08750542,  0.12748615, -0.10542529, -0.11433046, -0.13527544,
         0.11033006,  0.00808538,  0.04920774, -0.06920921,  0.12361796]],
      dtype=float32)

In [38]:
test = pd.DataFrame({'a':range(4),'b':range(4,8)})
test
for a,b in zip(test.a, test.b): 
    print(a,b) 

0 4
1 5
2 6
3 7


In [17]:
T1=model.get_embeddings(['fever'],embedding_type='entity')

In [18]:
T1

array([[ 0.06438797, -0.12558475,  0.08747587,  0.14357647, -0.07907126,
         0.13641204,  0.00495911,  0.05627691,  0.0819731 , -0.08718582]],
      dtype=float32)

In [19]:
R1=model.get_embeddings(['include'], embedding_type='relation')

In [20]:
R1

array([[ 0.2897198 , -0.24677382, -0.3173578 , -0.29528517,  0.34402892,
         0.29501817,  0.3511118 ,  0.29920095, -0.3037056 , -0.24746387]],
      dtype=float32)

In [21]:
df3=np.add(H1,R1)-T1

In [22]:
df3

array([[ 0.31283724,  0.00629708, -0.510259  , -0.55319214,  0.28782475,
         0.26893622,  0.35423806,  0.29213178, -0.45488793, -0.03666008]],
      dtype=float32)

In [23]:
from sklearn import preprocessing
normalized = preprocessing.normalize(df3)
print("Normalized Data = ", normalized)

Normalized Data =  [[ 0.28109998  0.00565824 -0.45849332 -0.49707094  0.258625    0.2416527
   0.3183007   0.26249507 -0.40873966 -0.03294092]]


In [24]:
abs(normalized)

array([[0.28109998, 0.00565824, 0.45849332, 0.49707094, 0.258625  ,
        0.2416527 , 0.3183007 , 0.26249507, 0.40873966, 0.03294092]],
      dtype=float32)

In [25]:
from statistics import mean

In [26]:
ar=abs(normalized)


In [27]:
for i in ar:
  Score=mean(i)

In [28]:
print(Score)

0.27650765


Getting Score for negative triple for 1 triple


In [32]:
HC1=model.get_embeddings(['Management'],embedding_type='entity')

In [33]:
HC1

array([[-0.09443514,  0.02835004,  0.07051636, -0.09715363, -0.09348316,
         0.04154861,  0.12599345,  0.11656423,  0.03365684,  0.02225728]],
      dtype=float32)

In [30]:
df4=np.add(HC1,R1)-(T1)

In [31]:
from sklearn import preprocessing
normalized1 = preprocessing.normalize(df4)
print("Normalized Data = ", normalized1)


Normalized Data =  [[ 0.12662159 -0.0898069  -0.32339847 -0.518509    0.31885165  0.19361766
   0.4567258   0.34774733 -0.3405248  -0.13351297]]


In [32]:
from statistics import mean
ar1=abs(normalized1)
for i in ar1:
  SC1=mean(i)
print(SC1)  

0.2849316


Getting Score for negative triple for 2 triple

In [33]:
HC2=model.get_embeddings(['The World Health Organization'],embedding_type='entity')

In [34]:
df5=np.add(HC2,R1)-(T1)

In [35]:
from sklearn import preprocessing
normalized2 = preprocessing.normalize(df5)
print("Normalized Data = ", normalized2)


Normalized Data =  [[ 0.08242828 -0.15542758 -0.4877485  -0.3263167   0.49526325  0.23497596
   0.18913415  0.15291385 -0.44927463 -0.25132674]]


In [36]:
from statistics import mean
ar2=abs(normalized2)
for i in ar2:
  SC2=mean(i)
print(SC2)  

0.28248096


Getting Score for negative triple for 3 triple

In [41]:
HC3=model.get_embeddings(['Cardiovascular complications'],embedding_type='entity')

In [42]:
df6=np.add(HC3,R1)-(T1)

In [43]:
from sklearn import preprocessing
normalized3 = preprocessing.normalize(df6)
print("Normalized Data = ", normalized3)


Normalized Data =  [[ 0.12308562 -0.0748052  -0.26172292 -0.3583264   0.4615978   0.02832597
   0.47120547  0.27905053 -0.47065267 -0.21694389]]


In [44]:
from statistics import mean
ar3=abs(normalized3)
for i in ar3:
  SC3=mean(i)
print(SC3)  

0.27457166


Getting Score for negative triple for 4 triple

In [45]:
HC4=model.get_embeddings(['Wuhan Central Hospital'],embedding_type='entity')

In [46]:
df7=np.add(HC4,R1)-(T1)

In [47]:
from sklearn import preprocessing
normalized4 = preprocessing.normalize(df7)
print("Normalized Data = ", normalized4)


Normalized Data =  [[ 0.05849195 -0.22313695 -0.34948078 -0.5682964   0.41502368  0.02908172
   0.3066494   0.21386765 -0.43250966 -0.04198511]]


In [48]:
from statistics import mean
ar4=abs(normalized4)
for i in ar4:
  SC4=mean(i)
print(SC4)  

0.26385233


In [49]:
import numpy as np
from ampligraph.evaluation.metrics import rank_score
y_pred = np.array([SC1, SC2, Score,SC3,SC4])
y_true = np.array([0, 0, 1, 0,0])
rank_score(y_true, y_pred)

3

In [12]:
from ampligraph.evaluation import evaluate_performance

In [13]:
positives_filter =np.concatenate((Y_test,Y_train))

In [14]:
from ampligraph.utils import save_model, restore_model

In [15]:
save_model(model, './covid1_model.pkl')

In [ ]:
restored_model = restore_model('./covid_model.pkl')

In [ ]:
restored_model2 = restore_model('./covid2_model.pkl')

In [ ]:
restored_model3 = restore_model('./covid3_model.pkl')

In [16]:
ranks = evaluate_performance(Y_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 25/25 [00:00<00:00, 300.22it/s]


In [17]:
df = pd.DataFrame(data=ranks, columns=["Subject(corrupt)", "Object(corrupt)"])

In [18]:
df.to_csv('Test_triple_ranks.csv')

In [19]:
print(ranks)

[[1038 1116]
 [ 745  774]
 [1516 1606]
 [1781 1670]
 [1595 1436]
 [1615 1622]
 [1003 1034]
 [1055 1035]
 [1019  979]
 [ 947  931]
 [ 703  679]
 [1728 1654]
 [ 924  975]
 [ 541  527]
 [ 793  792]
 [1556 1533]
 [ 645  618]
 [1749 1565]
 [1249 1119]
 [   2   27]
 [1791 1792]
 [ 391  151]
 [  55  106]
 [1229 1309]
 [1744 1749]]


In [20]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
mr = mr_score(ranks)
mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))
hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.01
MR: 1084.26
Hits@10: 0.02
Hits@3: 0.02
Hits@1: 0.00


In [21]:
from ampligraph.discovery import discover_facts
from ampligraph.discovery import query_topn


In [22]:
ampligraph.discovery.query_topn(model, top_n=10, head='common symptoms', relation='include', tail=None, ents_to_consider=None, rels_to_consider=None)

(array([['common symptoms', 'include', 'cough'],
        ['common symptoms', 'include', 'muscle fatigue'],
        ['common symptoms', 'include', 'similar numbers'],
        ['common symptoms', 'include', 'Jennifer Ouelette'],
        ['common symptoms', 'include',
         'New Zealand Finance Minister Grant Robertson'],
        ['common symptoms', 'include', 'evidence'],
        ['common symptoms', 'include', 'Blood stasis'],
        ['common symptoms', 'include', 'examiner'],
        ['common symptoms', 'include', 'types'],
        ['common symptoms', 'include', 'ACE2 protein']], dtype='<U70'),
 array([0.0359018 , 0.03216413, 0.03080531, 0.02975412, 0.02901835,
        0.02885466, 0.02838279, 0.02799264, 0.0272607 , 0.026776  ],
       dtype=float32))

In [ ]:
from ampligraph.utils import create_tensorboard_visualizations

In [ ]:
create_tensorboard_visualizations(model,r"./")

In [ ]:
X_unseen = np.array([
    ['guidelines', 'recommend', 'medication'],
  ['black musician', 'wears', 'royal livery'],
   ['Common effects', 'include', 'chest infections'],
 #   ['pandemic', 'include', 'global social economic disruption']

])

In [ ]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))

In [ ]:
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=restored_model3, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)

100%|██████████| 3/3 [00:00<00:00, 57.72it/s]


In [ ]:
scores = model.predict(X_unseen)

In [ ]:
from scipy.special import expit
probs = expit(scores)

In [ ]:
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")

,statement,rank,score,prob
2,Common effects include chest infections,1083,0.004979,0.501245
1,black musician wears royal livery,12,0.007560,0.501890
0,guidelines recommend medication,2,0.016263,0.504066
